In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import sys

from keras.layers import LSTM
from keras.models import Sequential


def gen_train_data(feature_df):
    """
    把数据加工成
    feature_index：存放数据的位置，分类变量改成one_hot column 展开后index
    feature_value：存放数据值（分类变量改成1）
    :param feature_df:
    :return:
    """
    train_data = {}
    target_temp = np.log1p(feature_df[label_column].values)
    target_temp_nan = np.isnan(target_temp)

    target_temp[target_temp_nan] = 0
    ## 直接取对数后有的目标值就变成了nan，这里提前处理下nan值
    train_data['y_train'] = target_temp  # np.log1p(feature_df[label_column].values)
    #     train_data =
    # train_data['y_train'] = (feature_df[['sum_grp_lmt_sale_amt']]-feature_df[['sum_grp_lmt_sale_amt']].mean())/feature_df[['sum_grp_lmt_sale_amt']].std()
    # 连续变量df
    co_feature = feature_df[co_feature_column]
    # 离线变量df
    ca_feature = feature_df[ca_feature_column]
    co_feature = (co_feature - co_feature.mean()) / co_feature.std()
    lbd = defaultdict(LabelEncoder)
    ca_feature = ca_feature.apply(lambda x: lbd[x.name].fit_transform(x))

    feature_value = pd.concat([co_feature, ca_feature], axis=1)
    feature_index = feature_value.copy()
    # feature_index 从1开始
    # columns_index=list(zip(feature_value.columns,range(1,len(feature_value.columns)+1)))
    cnt = 1
    for c in feature_value.columns:
        if c in co_feature_column:
            feature_index[c] = cnt
            cnt += 1
        else:
            feature_index[c] += cnt
            feature_value[c] = 1
            cnt += lbd[c].classes_.shape[0]
    # feature_index是特征的一个序号，主要用于通过embedding_lookup选择我们的embedding
    train_data['xi'] = feature_index.values
    # feature_value是对应的特征值，如果是离散特征的话，就是1，如果不是离散特征的话，就保留原来的特征值。
    train_data['xv'] = feature_value.fillna(0).values
    train_data['feat_dim'] = cnt
    train_data['xc'] = co_feature.fillna(0).values
    # train_data['y_train']=np.ones((len(feature_df),1))
    return train_data


def to_supervised(data_dict, n_input, n_out, batch_sizes):
    #     data = train
    data_train = data_dict['xv']
    data_index = data_dict['xi']
    data_y = data_dict['y_train']
    data_con = data_dict['xc']
    X_data, X_index, y, X_continuous = list(), list(), list(), list()
    in_start = 0
    batch_index = []
    for i in range(len(data_train)):
        in_end = in_start + n_input
        out_end = in_end + n_out
        if out_end <= len(data_train):
            X_data.append(data_train[in_start:in_end, :])  # 使用几个特征
            X_index.append(data_index[in_start:in_end, :])
            y.append(data_y[in_end:out_end, 0])
            X_continuous.append(data_con[in_start:in_end, :])
            if i % batch_sizes == 0:
                batch_index.append(i)
        in_start += 1
    return np.array(X_data), np.array(X_index, dtype=np.int32), np.array(y), np.array(X_continuous), batch_index





batch_sizes = 55
time_steps = 10
con_features_size = 19
ca_features_size = 5
feature_sizes = con_features_size + ca_features_size
embedding_size = 5
rnn_unit = 5
epoch = 50

# 生成模型使用的数据


co_feature_column = list('abcdefghijklmnopqrs')
ca_feature_column = list('vwxyz')

df_co = pd.DataFrame(np.random.normal(0, 1, size=(600, len(co_feature_column))), columns=co_feature_column)
df_ca = pd.DataFrame(np.random.randint(1, 30, size=(600, len(ca_feature_column))), columns=ca_feature_column)
# pd.read_csv('./lstmfm_data.csv', sep='\t').fillna(0)
df = pd.concat([df_co, df_ca], axis=1)
df['target'] = 0
label_column = ['target']
data_dict = gen_train_data(df)

n_dim = data_dict['feat_dim']

data_train_, data_index_, data_y_, data_con_, batch_index_ = to_supervised(data_dict,
                                                                           10, 1,
                                                                           80)
# data_dict = train_data

# 用于lstmfm最后一层的输入，当作全连接层
weights = {
    'out': tf.Variable(tf.random_normal([rnn_unit, 1])),
    'feature_weight': tf.Variable(tf.random_normal([n_dim, embedding_size], 0, 1.0),
                                  name='feature_weight'),
    'feature_first': tf.Variable(tf.random_normal([n_dim, 1], 0, 1.0), name='feature_first')
}
biases = {
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}

feat_index = tf.placeholder(tf.int32, shape=[None, time_steps, feature_sizes],
                                 name='feature_index')
feat_value = tf.placeholder(tf.float32, shape=[None, time_steps, feature_sizes],
                                 name='feature_value')
label = tf.placeholder(tf.float32, shape=[None, None, None], name='label')
continuous_part = tf.placeholder(tf.float32, shape=[None, time_steps, con_features_size],
                                      name='continuous_feautre')



def build_model(batch_size_changes):
    # def lstmfm():
    """
    构建网络结构 lstmfm
    :return:
    """

    # One-hot编码后的输入层与Dense embeddings层的权值定义，即DNN的输入embedding。
    # 注：Dense embeddings层的神经元个数由field_size和决定
    # （61*256）
    #         weights['feature_weight'] = tf.Variable(tf.random_normal([n_dim, embedding_size], 0, 1),
    #                                                name='feature_weight')

    #         # FM部分中一次项的权值定义
    #         # shape (61,1)
    #         weights['feature_first'] = tf.Variable(tf.random_normal([n_dim, 1], 0, 1.0),
    #                                               name='feature_first')

    # embedding_part
    # shape (?,39,256) 当前特征的权重

    feat_index_ = tf.reshape(feat_index, [-1, feature_sizes])
    feat_value_ = tf.reshape(feat_value, [-1, feature_sizes])
    print("feat_value shape is", feat_value_.shape)
    print("feat_index shape is ", feat_index_.shape)

    embedding_index = tf.nn.embedding_lookup(weights['feature_weight'], feat_index_)  # Batch*F*K
    print('embedding_part shape is', embedding_index.shape)

    # shape (?,39,256)  BFK * BF1=BFK
    embedding_part = tf.multiply(embedding_index, tf.reshape(feat_value_, [-1, feature_sizes, 1]))
    # [Batch*F*1] * [Batch*F*K] = [Batch*F*K],用到了broadcast的属性
    print('embedding_part:', embedding_part)

    """
    网络传递结构
    """
    # FM部分
    # 一阶特征
    # shape (?,39,1)

    embedding_first = tf.nn.embedding_lookup(weights['feature_first'], feat_index_)  # bacth*F*1
    embedding_first = tf.multiply(tf.reshape(feat_value_, [-1, feature_sizes, 1]), embedding_first)
    print("embedding_first shape is", embedding_first.shape)
    # shape （？,39）
    first_order = tf.reduce_sum(embedding_first, 2)
    print('first_order:', first_order.shape)

    # 二阶特征 和的平方-平方的和
    sum_second_order = tf.reduce_sum(embedding_part, 1)
    sum_second_order_square = tf.square(sum_second_order)
    print('sum_square_second_order:', sum_second_order_square)

    square_second_order = tf.square(embedding_part)
    square_second_order_sum = tf.reduce_sum(square_second_order, 1)
    print('square_sum_second_order:', square_second_order_sum)
    ## 嵌入的部分特征
    # 1/2*((a+b)^2 - a^2 - b^2)=ab
    second_order = 0.5 * tf.subtract(sum_second_order_square, square_second_order_sum)

    # FM部分的输出(39+256) ：用的是concat
    fm_part = tf.concat([first_order, second_order], axis=1)
    print('fm_part:', fm_part)
    fm_part = tf.reshape(fm_part, [-1, time_steps, feature_sizes + embedding_size])
    embedding_part = tf.reshape(embedding_part, [-1, time_steps, embedding_size * feature_sizes])
    print("fm_part shape is", fm_part.shape)
    print("embedding shape is", embedding_part.shape)
    print("continuous_part shape is", continuous_part.shape)

    din_all = tf.concat([fm_part, embedding_part, continuous_part], axis=2)
    print(".........din_all shape is", din_all.shape)
    print("..........din_all shape is", din_all.shape)

    # model = Sequential()
    # model.add(LSTM(rnn_unit, input_shape=(time_steps, din_all.shape[0])))
    # model.summary()

    # 改用lstm模型
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    # init_state = cell.zero_state(batch_sizes, dtype=tf.float32)

    init_state = cell.zero_state(batch_size_changes,
                                 dtype=tf.float32)  #########################################################################
    output_rnn, final_states = tf.nn.dynamic_rnn(cell, 
                                                 din_all, 
                                                 initial_state=init_state,
                                                 dtype=tf.float32)  # output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    print(output_rnn.shape)
    output_temp = tf.reshape(output_rnn, [-1, time_steps, rnn_unit])  # 作为输出层的输入
    output = output_temp[:, -1, :]

    print(".......output shape", output.shape)
    print(":::::::::final_states shape is", final_states)

    w_out = weights['out']
    b_out = biases['out']
    print("----w_out shape is", w_out.shape)
    print("----b_out shape is", b_out.shape)
    print("............weights out shape is", weights['out'].shape)
    pred = tf.matmul(output, w_out) + b_out

    print("------pred shape is ", pred.shape)

    #     return pred, final_states
    # loss部分
    # out = tf.nn.sigmoid(out)
    #
    # loss = -tf.reduce_mean(
    #     label * tf.log(out + 1e-24) + (1 - label) * tf.log(1 - out + 1e-24))
    # 改成平方差损失函数
    # loss = tf.reduce_mean((out - label) ** 2)
    #     loss = tf.nn.l2_loss(tf.subtract(label, out))

    # 正则：sum(w^2)/2*l2_reg_rate
    # 这边只加了weight，有需要的可以加上bias部分
    # loss += tf.contrib.layers.l2_regularizer(l2_reg_rate)(weight["last_layer"])

    #     global_step = tf.Variable(0, trainable=False)
    # opt = tf.train.GradientDescentOptimizer(learning_rate)
    # trainable_params = tf.trainable_variables()
    # print(trainable_params)
    # gradients = tf.gradients(loss, trainable_params)
    # clip_gradients, _ = tf.clip_by_global_norm(gradients, 5)
    # train_op = opt.apply_gradients(
    #     zip(clip_gradients, trainable_params), global_step=global_step)
    # optimizer
    #     if optimizer_type == "adam":
    #         optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.999,
    #                                                 epsilon=1e-8).minimize(loss)
    #     elif optimizer_type == "adagrad":
    #         optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate,
    #                                                    initial_accumulator_value=1e-8).minimize(loss)
    #     elif optimizer_type == "gd":
    #         optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    #     elif optimizer_type == "momentum":
    #         optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.95).minimize(
    #             loss)
    return pred

def train():
    global batch_sizes
    data_train, data_index, data_y, data_con, batch_index = to_supervised(data_dict,
                                                                          time_steps, 1,
                                                                          batch_sizes)
    print(type(batch_index))
    print(np.array(data_train).shape)  # 3785  15  7
    with tf.variable_scope("sec_lstm"):
        pred = build_model(batch_sizes)
    loss = tf.reduce_mean(tf.square(tf.reshape(pred, [-1]) - tf.reshape(label, [-1])))
    
    print(".......loss",loss.shape)
    print(".......loss",loss.shape)
    train_op = tf.train.AdamOptimizer(0.01).minimize(loss)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        tf.get_variable_scope().reuse_variables()
        sess.run(tf.global_variables_initializer())

        # 重复训练200次
        for i in range(epoch):  # epoch
            # 每次进行训练的时候，每个batch训练batch_sizes个样本
            for step in range(len(batch_index) - 1):
                # print(data_train[batch_index[step]:batch_index[step + 1]])
                _, loss_ = sess.run([train_op, loss],
                                    feed_dict={feat_value: data_train[batch_index[step]:batch_index[step + 1]],
                                               feat_index: data_index[batch_index[step]:batch_index[step + 1]],
                                               continuous_part: data_con[batch_index[step]:batch_index[step + 1]],
                                               label: np.array(
                                                   data_y[batch_index[step]:batch_index[step + 1]]).reshape(-1,
                                                                                                            batch_sizes,
                                                                                                            1)})
            print(i, loss_)

            
        print("保存模型：", saver.save(sess, 'model_save1/modle.ckpt'))
        print("Well Done!")
    # pred_ = sess.run([pred],
    #          feed_dict={feat_value: data_train,
    #                     feat_index: data_index,
    #                     continuous_part: data_con
    #                     }
    #          )
    
test_value = data_dict['xv'][0:10, 0:24].reshape(1, 10, 24)
print("...............test_value shape is", test_value.shape)
test_index = data_dict['xi'][0:10, 0:24].reshape(1, 10, 24)
con_values = data_dict['xc'][0:10, :].reshape(1, 10, 19)
def prediction(test_value,test_index,con_values):
    with tf.variable_scope("sec_lstm", reuse=tf.AUTO_REUSE):
        pred= build_model(1)
    saver = tf.train.Saver(tf.global_variables())

    with tf.Session() as sess:
        tf.get_variable_scope().reuse_variables()
        saver.restore(sess, 'model_save1/modle.ckpt')
        # I run the code in windows 10,so use  'model_save1\\modle.ckpt'
        # if you run it in Linux,please use  'model_save1/modle.ckpt'
        predict = []

        next_seq = sess.run(pred, feed_dict={feat_value: test_value,
                                       feat_index: test_index,
                                       continuous_part: con_values
                                       })
        predict.append(next_seq[-1])
        print(predict)
#         plt.figure()
#         plt.plot(list(range(len(data))),data, color='b')
#         plt.plot(list(range(time_step + 1, np.shape(train_x)[0] + 1 + time_step)), [ value*s+m for value in predict], color='r')
#         plt.show()

# prediction()


# 加工训练数据



train()

prediction(test_value,test_index,con_values)


/home/gaoxc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gaoxc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gaoxc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gaoxc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

...............test_value shape is (1, 10, 24)
<class 'list'>
(590, 10, 24)
feat_value shape is (?, 24)
feat_index shape is  (?, 24)
embedding_part shape is (?, 24, 5)
embedding_part: Tensor("sec_lstm/Mul:0", shape=(?, 24, 5), dtype=float32)
embedding_first shape is (?, 24, 1)
first_order: (?, 24)
sum_square_second_order: Tensor("sec_lstm/Square:0", shape=(?, 5), dtype=float32)
square_sum_second_order: Tensor("sec_lstm/Sum_2:0", shape=(?, 5), dtype=float32)
fm_part: Tensor("sec_lstm/concat:0", shape=(?, 29), dtype=float32)
fm_part shape is (?, 10, 29)
embedding shape is (?, 10, 120)
continuous_part shape is (?, 10, 19)
.........din_all shape is (?, 10, 168)
..........din_all shape is (?, 10, 168)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Using TensorFlow backend.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(55, 10, 5)
.......output shape (55, 5)
:::::::::final_states shape is LSTMStateTuple(c=<tf.Tensor 'sec_lstm/rnn/while/Exit_3:0' shape=(55, 5) dtype=float32>, h=<tf.Tensor 'sec_lstm/rnn/while/Exit_4:0' shape=(55, 5) dtype=float32>)
----w_out shape is (5, 1)
----b_out shape is (1,)
............weights out shape is (5, 1)
------pred shape is  (55, 1)
.......loss ()
.......loss ()
0 0.15202598
1 0.027855374
2 0.020780519
3 0.012039938
4 0.008758754
5 0.004854136
6 0.0031111573
7 0.0025869824
8 0.0020096526
9 0.0015763871
10 0.0013632715
11 0.0011342533
12 0.0009869587
13 0.00084372517
14 0.00078893075
15 0.00070082216
16 0.0006011833
17 0.00058343617
18 0.0005235785
19 0.0005353136
20 0.00052509014
21 0.0005211614
22 0.0004964854
23 0.00038567002
24 0.00039

In [2]:
prediction(test_value,test_index,con_values)

feat_value shape is (?, 24)
feat_index shape is  (?, 24)
embedding_part shape is (?, 24, 5)
embedding_part: Tensor("sec_lstm_2/Mul:0", shape=(?, 24, 5), dtype=float32)
embedding_first shape is (?, 24, 1)
first_order: (?, 24)
sum_square_second_order: Tensor("sec_lstm_2/Square:0", shape=(?, 5), dtype=float32)
square_sum_second_order: Tensor("sec_lstm_2/Sum_2:0", shape=(?, 5), dtype=float32)
fm_part: Tensor("sec_lstm_2/concat:0", shape=(?, 29), dtype=float32)
fm_part shape is (?, 10, 29)
embedding shape is (?, 10, 120)
continuous_part shape is (?, 10, 19)
.........din_all shape is (?, 10, 168)
..........din_all shape is (?, 10, 168)
(1, 10, 5)
.......output shape (1, 5)
:::::::::final_states shape is LSTMStateTuple(c=<tf.Tensor 'sec_lstm_2/rnn/while/Exit_3:0' shape=(1, 5) dtype=float32>, h=<tf.Tensor 'sec_lstm_2/rnn/while/Exit_4:0' shape=(1, 5) dtype=float32>)
----w_out shape is (5, 1)
----b_out shape is (1,)
............weights out shape is (5, 1)
------pred shape is  (1, 1)
INFO:tensorf